# Week 7: Advanced EDA with Business Intelligence - Part 3: Time Series Patterns in Order Data

## Learning Objectives
By the end of this session, you will be able to:
- Conduct comprehensive time series analysis for business intelligence
- Identify seasonal patterns, trends, and cyclical behaviors in order data
- Apply advanced temporal analytics for forecasting and planning
- Create time-based customer and product insights
- Build predictive models for business forecasting

## Business Context
Completing our advanced EDA trilogy, we focus on **temporal patterns and time series analysis** to understand:
- **Seasonal Business Cycles**: When do customers buy the most?
- **Growth Trends**: How is the business evolving over time?
- **Demand Forecasting**: What can we expect in future periods?
- **Operational Planning**: How to optimize inventory and staffing?

**Key Business Questions:**
- What are the seasonal patterns in our sales data?
- How do different customer segments behave over time?
- Which time periods drive the highest revenue and why?
- How can we forecast future demand and plan accordingly?

## 1. Environment Setup and Secure Data Connection

In [ ]:
# Essential imports for time series analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Time series specific libraries
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats
from scipy.signal import find_peaks

# Advanced analytics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Interactive visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

# Database connection (secure)
import os
from sqlalchemy import create_engine

# Display and plotting settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 8)

print("✅ Environment setup complete for time series analysis!")

In [ ]:
# Secure Database Connection Using Environment Variables
# Best practice: Never expose credentials in code

# Set up environment variables securely
if 'SUPABASE_DB_HOST' not in os.environ:
    # For educational purposes only - in production, set these at system level
    os.environ['SUPABASE_DB_HOST'] = 'aws-0-us-east-1.pooler.supabase.com'
    os.environ['SUPABASE_DB_PORT'] = '6543'
    os.environ['SUPABASE_DB_NAME'] = 'postgres'
    os.environ['SUPABASE_DB_USER'] = 'postgres.pzykoxdiwsyclwfqfiii'
    os.environ['SUPABASE_DB_PASSWORD'] = 'L3tMeQuery123!'

# Construct secure database URL
DATABASE_URL = f"postgresql://{os.environ['SUPABASE_DB_USER']}:{os.environ['SUPABASE_DB_PASSWORD']}@{os.environ['SUPABASE_DB_HOST']}:{os.environ['SUPABASE_DB_PORT']}/{os.environ['SUPABASE_DB_NAME']}"

# Create database engine
engine = create_engine(DATABASE_URL)

# Test connection
try:
    with engine.connect() as conn:
        result = conn.execute("SELECT 1 as connection_test")
        print("✅ Secure database connection established!")
except Exception as e:
    print(f"❌ Connection failed: {e}")

print("🔒 Security Note: Database credentials loaded from environment variables")

## 2. Comprehensive Time Series Data Loading

Load temporal data optimized for time series analysis.

In [ ]:
# Comprehensive Time Series Dataset Loading
print("🔄 Loading comprehensive time series dataset...")

# Time series optimized query
timeseries_query = """
WITH daily_orders AS (
    SELECT 
        DATE(o.order_purchase_timestamp) as order_date,
        o.order_id,
        o.customer_id,
        o.order_purchase_timestamp,
        EXTRACT(YEAR FROM o.order_purchase_timestamp) as order_year,
        EXTRACT(MONTH FROM o.order_purchase_timestamp) as order_month,
        EXTRACT(DAY FROM o.order_purchase_timestamp) as order_day,
        EXTRACT(DOW FROM o.order_purchase_timestamp) as order_dow,
        EXTRACT(WEEK FROM o.order_purchase_timestamp) as order_week,
        EXTRACT(QUARTER FROM o.order_purchase_timestamp) as order_quarter,
        EXTRACT(HOUR FROM o.order_purchase_timestamp) as order_hour,
        
        c.customer_state,
        c.customer_city,
        
        oi.product_id,
        oi.price,
        oi.freight_value,
        (oi.price + oi.freight_value) as total_order_value,
        
        p.product_category_name,
        COALESCE(pt.product_category_name_english, p.product_category_name) as category_english,
        
        r.review_score,
        
        -- Calculate days since business start for trend analysis
        DATE_PART('day', o.order_purchase_timestamp - 
                 (SELECT MIN(order_purchase_timestamp) FROM olist_sales_data_set.olist_orders_dataset)) as days_since_start
        
    FROM olist_sales_data_set.olist_orders_dataset o
    JOIN olist_sales_data_set.olist_customers_dataset c ON o.customer_id = c.customer_id
    JOIN olist_sales_data_set.olist_order_items_dataset oi ON o.order_id = oi.order_id
    JOIN olist_sales_data_set.olist_products_dataset p ON oi.product_id = p.product_id
    LEFT JOIN olist_sales_data_set.product_category_name_translation pt 
        ON p.product_category_name = pt.product_category_name
    LEFT JOIN olist_sales_data_set.olist_order_reviews_dataset r ON o.order_id = r.order_id
    
    WHERE o.order_status = 'delivered'
    AND oi.price > 0
    AND o.order_purchase_timestamp IS NOT NULL
)
SELECT * FROM daily_orders
ORDER BY order_purchase_timestamp
LIMIT 35000;
"""

# Load the time series data
ts_df = pd.read_sql(timeseries_query, engine)

# Data preprocessing for time series analysis
ts_df['order_purchase_timestamp'] = pd.to_datetime(ts_df['order_purchase_timestamp'])
ts_df['order_date'] = pd.to_datetime(ts_df['order_date'])
ts_df['category_clean'] = ts_df['category_english'].fillna('Unknown').str.title()

# Create additional time-based features
ts_df['is_weekend'] = ts_df['order_dow'].isin([0, 6])  # Sunday=0, Saturday=6
ts_df['month_name'] = ts_df['order_purchase_timestamp'].dt.month_name()
ts_df['day_name'] = ts_df['order_purchase_timestamp'].dt.day_name()
ts_df['is_holiday_season'] = ts_df['order_month'].isin([11, 12])  # Nov-Dec

# Define business periods
def get_business_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

ts_df['business_period'] = ts_df['order_hour'].apply(get_business_period)

# Calculate analysis period
analysis_start = ts_df['order_date'].min()
analysis_end = ts_df['order_date'].max()
total_days = (analysis_end - analysis_start).days

print(f"✅ Time series dataset loaded successfully!")
print(f"   📊 Total records: {len(ts_df):,}")
print(f"   📅 Analysis period: {analysis_start.date()} to {analysis_end.date()} ({total_days} days)")
print(f"   🛒 Unique orders: {ts_df['order_id'].nunique():,}")
print(f"   👥 Unique customers: {ts_df['customer_id'].nunique():,}")
print(f"   🏷️ Product categories: {ts_df['category_clean'].nunique()}")

# Display sample data
print("\n📋 Sample Time Series Data:")
display(ts_df[['order_date', 'order_hour', 'day_name', 'category_clean', 
              'total_order_value', 'business_period']].head())

## 3. Time Series Aggregation and Basic Patterns

Create different temporal aggregations to identify patterns at various time scales.

In [ ]:
# Time Series Aggregation and Pattern Analysis
print("📊 Time Series Aggregation and Pattern Analysis")
print("=" * 50)

def create_time_aggregations(data):
    """
    Create multiple time-based aggregations for analysis
    """
    aggregations = {}
    
    # Daily aggregation
    daily_agg = data.groupby('order_date').agg({
        'order_id': 'nunique',
        'customer_id': 'nunique',
        'total_order_value': ['sum', 'mean', 'count'],
        'review_score': 'mean'
    }).reset_index()
    
    # Flatten column names
    daily_agg.columns = ['date', 'unique_orders', 'unique_customers', 
                        'total_revenue', 'avg_order_value', 'total_items', 'avg_review']
    
    # Weekly aggregation
    weekly_agg = data.groupby([data['order_purchase_timestamp'].dt.to_period('W')]).agg({
        'order_id': 'nunique',
        'customer_id': 'nunique',
        'total_order_value': ['sum', 'mean'],
        'review_score': 'mean'
    }).reset_index()
    
    weekly_agg.columns = ['week', 'unique_orders', 'unique_customers', 
                         'total_revenue', 'avg_order_value', 'avg_review']
    weekly_agg['week_start'] = weekly_agg['week'].dt.start_time
    
    # Monthly aggregation
    monthly_agg = data.groupby([data['order_purchase_timestamp'].dt.to_period('M')]).agg({
        'order_id': 'nunique',
        'customer_id': 'nunique',
        'total_order_value': ['sum', 'mean'],
        'review_score': 'mean',
        'category_clean': 'nunique'
    }).reset_index()
    
    monthly_agg.columns = ['month', 'unique_orders', 'unique_customers', 
                          'total_revenue', 'avg_order_value', 'avg_review', 'categories_sold']
    monthly_agg['month_start'] = monthly_agg['month'].dt.start_time
    
    # Hourly patterns
    hourly_agg = data.groupby('order_hour').agg({
        'order_id': 'nunique',
        'total_order_value': ['sum', 'mean'],
        'customer_id': 'nunique'
    }).reset_index()
    
    hourly_agg.columns = ['hour', 'unique_orders', 'total_revenue', 'avg_order_value', 'unique_customers']
    
    # Day of week patterns
    dow_agg = data.groupby(['order_dow', 'day_name']).agg({
        'order_id': 'nunique',
        'total_order_value': ['sum', 'mean'],
        'customer_id': 'nunique'
    }).reset_index()
    
    dow_agg.columns = ['dow_num', 'day_name', 'unique_orders', 'total_revenue', 'avg_order_value', 'unique_customers']
    
    aggregations['daily'] = daily_agg
    aggregations['weekly'] = weekly_agg
    aggregations['monthly'] = monthly_agg
    aggregations['hourly'] = hourly_agg
    aggregations['day_of_week'] = dow_agg
    
    return aggregations

# Create time aggregations
time_aggregations = create_time_aggregations(ts_df)

# Display aggregation summaries
print(f"📈 Time Series Aggregation Summary:")
for period, agg_data in time_aggregations.items():
    if period not in ['hourly', 'day_of_week']:
        print(f"   • {period.title()}: {len(agg_data)} periods")
        print(f"     Revenue range: R$ {agg_data['total_revenue'].min():.2f} - R$ {agg_data['total_revenue'].max():.2f}")
        print(f"     Average daily revenue: R$ {agg_data['total_revenue'].mean():.2f}")
        print()

# Identify peak performance periods
daily_data = time_aggregations['daily']
peak_revenue_day = daily_data.loc[daily_data['total_revenue'].idxmax()]
peak_orders_day = daily_data.loc[daily_data['unique_orders'].idxmax()]

print(f"🏆 Peak Performance Analysis:")
print(f"   • Highest revenue day: {peak_revenue_day['date'].date()} (R$ {peak_revenue_day['total_revenue']:,.2f})")
print(f"   • Highest order volume day: {peak_orders_day['date'].date()} ({peak_orders_day['unique_orders']:,} orders)")

# Basic trend analysis
monthly_data = time_aggregations['monthly']
if len(monthly_data) > 1:
    revenue_growth = ((monthly_data['total_revenue'].iloc[-1] - monthly_data['total_revenue'].iloc[0]) / 
                     monthly_data['total_revenue'].iloc[0] * 100)
    order_growth = ((monthly_data['unique_orders'].iloc[-1] - monthly_data['unique_orders'].iloc[0]) / 
                   monthly_data['unique_orders'].iloc[0] * 100)
    
    print(f"\n📊 Overall Growth Trends:")
    print(f"   • Revenue growth: {revenue_growth:+.1f}% (first to last month)")
    print(f"   • Order volume growth: {order_growth:+.1f}% (first to last month)")

# Display key aggregation data
print(f"\n📋 Monthly Performance Overview:")
display(monthly_data[['month', 'unique_orders', 'total_revenue', 'avg_order_value', 'avg_review']].head(10))

In [ ]:
# Comprehensive Temporal Pattern Visualization
print("📊 Creating Comprehensive Temporal Pattern Visualizations")
print("=" * 60)

# Create comprehensive temporal visualization dashboard
fig = plt.figure(figsize=(20, 16))

# 1. Daily Revenue Trend
plt.subplot(3, 3, 1)
daily_data = time_aggregations['daily']
plt.plot(daily_data['date'], daily_data['total_revenue'], alpha=0.7, color='blue')
plt.plot(daily_data['date'], daily_data['total_revenue'].rolling(7).mean(), 
         color='red', linewidth=2, label='7-day MA')
plt.title('Daily Revenue Trend', fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Revenue (R$)')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 2. Monthly Revenue and Orders
plt.subplot(3, 3, 2)
monthly_data = time_aggregations['monthly']
ax1 = plt.gca()
ax1.bar(range(len(monthly_data)), monthly_data['total_revenue'], 
        alpha=0.7, color='lightblue', label='Revenue')
ax1.set_xlabel('Month')
ax1.set_ylabel('Revenue (R$)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(range(len(monthly_data)), monthly_data['unique_orders'], 
         color='red', marker='o', linewidth=2, label='Orders')
ax2.set_ylabel('Number of Orders', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Monthly Revenue vs Orders', fontweight='bold')
plt.xticks(range(len(monthly_data)), 
           [str(m)[:7] for m in monthly_data['month']], rotation=45)

# 3. Hourly Distribution
plt.subplot(3, 3, 3)
hourly_data = time_aggregations['hourly']
plt.bar(hourly_data['hour'], hourly_data['unique_orders'], 
        color='lightgreen', alpha=0.7)
plt.title('Orders by Hour of Day', fontweight='bold')
plt.xlabel('Hour')
plt.ylabel('Number of Orders')
plt.grid(True, alpha=0.3)

# 4. Day of Week Analysis
plt.subplot(3, 3, 4)
dow_data = time_aggregations['day_of_week']
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_ordered = dow_data.set_index('day_name').reindex(day_order).reset_index()
plt.bar(dow_ordered['day_name'], dow_ordered['total_revenue'], 
        color='coral', alpha=0.7)
plt.title('Revenue by Day of Week', fontweight='bold')
plt.xlabel('Day')
plt.ylabel('Total Revenue (R$)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 5. Weekend vs Weekday Analysis
plt.subplot(3, 3, 5)
weekend_analysis = ts_df.groupby('is_weekend').agg({
    'total_order_value': ['sum', 'mean'],
    'order_id': 'nunique'
})
weekend_analysis.columns = ['total_revenue', 'avg_order_value', 'unique_orders']
weekend_analysis.index = ['Weekday', 'Weekend']

weekend_analysis['avg_order_value'].plot(kind='bar', color=['skyblue', 'orange'])
plt.title('Average Order Value: Weekday vs Weekend', fontweight='bold')
plt.ylabel('Average Order Value (R$)')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)

# 6. Business Period Analysis
plt.subplot(3, 3, 6)
period_analysis = ts_df.groupby('business_period').agg({
    'order_id': 'nunique',
    'total_order_value': 'sum'
})
period_order = ['Morning', 'Afternoon', 'Evening', 'Night']
period_analysis = period_analysis.reindex(period_order)

period_analysis['order_id'].plot(kind='bar', color='lightpink')
plt.title('Orders by Business Period', fontweight='bold')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 7. Customer Acquisition Over Time
plt.subplot(3, 3, 7)
customer_first_order = ts_df.groupby('customer_id')['order_date'].min().reset_index()
customer_acquisition = customer_first_order.groupby('order_date').size().cumsum()
plt.plot(customer_acquisition.index, customer_acquisition.values, color='purple', linewidth=2)
plt.title('Cumulative Customer Acquisition', fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Total Customers')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 8. Revenue Distribution Over Time
plt.subplot(3, 3, 8)
daily_data_clean = daily_data.dropna()
plt.hist(daily_data_clean['total_revenue'], bins=30, alpha=0.7, color='gold', edgecolor='black')
plt.axvline(daily_data_clean['total_revenue'].mean(), color='red', linestyle='--', 
           label=f'Mean: R${daily_data_clean["total_revenue"].mean():.2f}')
plt.title('Daily Revenue Distribution', fontweight='bold')
plt.xlabel('Daily Revenue (R$)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# 9. Order Value vs Volume Correlation
plt.subplot(3, 3, 9)
plt.scatter(daily_data['unique_orders'], daily_data['avg_order_value'], 
           alpha=0.6, color='teal', s=50)
correlation = daily_data['unique_orders'].corr(daily_data['avg_order_value'])
plt.title(f'Orders vs Avg Order Value\n(Correlation: {correlation:.3f})', fontweight='bold')
plt.xlabel('Number of Orders')
plt.ylabel('Average Order Value (R$)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary insights
print(f"\n💡 Key Temporal Pattern Insights:")

# Peak hour
peak_hour = hourly_data.loc[hourly_data['unique_orders'].idxmax(), 'hour']
print(f"   • Peak ordering hour: {peak_hour}:00")

# Best day of week
best_dow = dow_data.loc[dow_data['total_revenue'].idxmax(), 'day_name']
print(f"   • Highest revenue day: {best_dow}")

# Weekend vs weekday
weekday_avg = weekend_analysis.loc['Weekday', 'avg_order_value']
weekend_avg = weekend_analysis.loc['Weekend', 'avg_order_value']
weekend_premium = ((weekend_avg - weekday_avg) / weekday_avg * 100)
print(f"   • Weekend vs weekday order value: {weekend_premium:+.1f}% difference")

# Revenue volatility
revenue_cv = daily_data['total_revenue'].std() / daily_data['total_revenue'].mean()
print(f"   • Daily revenue volatility (CV): {revenue_cv:.2f}")

## 4. Advanced Seasonal Decomposition and Trend Analysis

Apply advanced time series techniques to decompose and understand seasonal patterns.

In [ ]:
# Advanced Seasonal Decomposition Analysis
print("🔄 Advanced Seasonal Decomposition Analysis")
print("=" * 45)

def perform_seasonal_decomposition(daily_data, metric='total_revenue'):
    """
    Perform seasonal decomposition on time series data
    """
    # Prepare data for decomposition
    ts_data = daily_data.set_index('date')[metric].fillna(method='ffill')
    
    # Ensure we have enough data points
    if len(ts_data) < 14:  # Need at least 2 weeks for weekly seasonality
        print(f"Insufficient data for seasonal decomposition ({len(ts_data)} days)")
        return None
    
    # Determine period for decomposition
    # Use 7 days for weekly seasonality
    period = 7
    
    print(f"\n📊 Seasonal Decomposition for {metric}:")
    print(f"   • Data points: {len(ts_data)}")
    print(f"   • Period: {period} days (weekly seasonality)")
    print(f"   • Date range: {ts_data.index.min().date()} to {ts_data.index.max().date()}")
    
    try:
        # Perform decomposition
        decomposition = seasonal_decompose(ts_data, model='additive', period=period)
        
        # Plot decomposition
        fig, axes = plt.subplots(4, 1, figsize=(15, 12))
        
        # Original series
        decomposition.observed.plot(ax=axes[0], title='Original Time Series', color='blue')
        axes[0].grid(True, alpha=0.3)
        
        # Trend
        decomposition.trend.plot(ax=axes[1], title='Trend Component', color='red')
        axes[1].grid(True, alpha=0.3)
        
        # Seasonal
        decomposition.seasonal.plot(ax=axes[2], title='Seasonal Component', color='green')
        axes[2].grid(True, alpha=0.3)
        
        # Residual
        decomposition.resid.plot(ax=axes[3], title='Residual Component', color='orange')
        axes[3].grid(True, alpha=0.3)
        
        plt.suptitle(f'Seasonal Decomposition - {metric.replace("_", " ").title()}', 
                    fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Analyze components
        trend_slope = np.polyfit(range(len(decomposition.trend.dropna())), 
                                decomposition.trend.dropna(), 1)[0]
        seasonal_strength = decomposition.seasonal.std() / ts_data.std()
        residual_variance = decomposition.resid.var()
        
        print(f"\n📈 Decomposition Analysis Results:")
        print(f"   • Trend slope: {trend_slope:.2f} per day ({'Increasing' if trend_slope > 0 else 'Decreasing'})")
        print(f"   • Seasonal strength: {seasonal_strength:.3f} (higher = more seasonal)")
        print(f"   • Residual variance: {residual_variance:.2f}")
        
        # Seasonal pattern analysis
        seasonal_pattern = decomposition.seasonal.head(period)
        day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        
        print(f"\n📅 Weekly Seasonal Pattern:")
        for i, (day, value) in enumerate(zip(day_names, seasonal_pattern)):
            direction = "↑" if value > 0 else "↓" if value < 0 else "→"
            print(f"   • {day}: {value:+.2f} {direction}")
        
        return decomposition
        
    except Exception as e:
        print(f"Error in seasonal decomposition: {e}")
        return None

# Perform decomposition on revenue
daily_data = time_aggregations['daily']
revenue_decomposition = perform_seasonal_decomposition(daily_data, 'total_revenue')

# Perform decomposition on order volume
print("\n" + "="*60)
orders_decomposition = perform_seasonal_decomposition(daily_data, 'unique_orders')

In [ ]:
# Advanced Trend Analysis and Stationarity Testing
print("📈 Advanced Trend Analysis and Stationarity Testing")
print("=" * 55)

def analyze_stationarity(ts_data, title="Time Series"):
    """
    Perform comprehensive stationarity analysis
    """
    print(f"\n🔍 Stationarity Analysis for {title}:")
    
    # Remove NaN values
    ts_clean = ts_data.dropna()
    
    if len(ts_clean) < 10:
        print("   Insufficient data for stationarity testing")
        return None
    
    # Augmented Dickey-Fuller test
    try:
        adf_result = adfuller(ts_clean)
        adf_statistic = adf_result[0]
        adf_pvalue = adf_result[1]
        adf_critical = adf_result[4]
        
        print(f"   📊 Augmented Dickey-Fuller Test:")
        print(f"     • Test Statistic: {adf_statistic:.4f}")
        print(f"     • p-value: {adf_pvalue:.4f}")
        print(f"     • Critical Values:")
        for key, value in adf_critical.items():
            print(f"       - {key}: {value:.4f}")
        
        is_stationary = adf_pvalue < 0.05
        print(f"     • Result: {'Stationary' if is_stationary else 'Non-stationary'} (α = 0.05)")
        
        return {
            'is_stationary': is_stationary,
            'adf_statistic': adf_statistic,
            'adf_pvalue': adf_pvalue
        }
        
    except Exception as e:
        print(f"   Error in stationarity testing: {e}")
        return None

def calculate_growth_metrics(daily_data):
    """
    Calculate various growth and trend metrics
    """
    metrics = {}
    
    # Prepare time series
    revenue_ts = daily_data.set_index('date')['total_revenue']
    orders_ts = daily_data.set_index('date')['unique_orders']
    
    # Calculate moving averages
    revenue_ma7 = revenue_ts.rolling(7).mean()
    revenue_ma30 = revenue_ts.rolling(30).mean()
    
    # Calculate growth rates
    revenue_growth_daily = revenue_ts.pct_change()
    revenue_growth_weekly = revenue_ts.pct_change(7)
    
    # Calculate volatility
    revenue_volatility = revenue_growth_daily.std()
    
    # Linear trend
    days_numeric = np.arange(len(revenue_ts))
    trend_slope, trend_intercept = np.polyfit(days_numeric, revenue_ts.fillna(method='ffill'), 1)
    
    metrics = {
        'avg_daily_revenue': revenue_ts.mean(),
        'revenue_volatility': revenue_volatility,
        'trend_slope': trend_slope,
        'max_daily_revenue': revenue_ts.max(),
        'min_daily_revenue': revenue_ts.min(),
        'revenue_range': revenue_ts.max() - revenue_ts.min(),
        'avg_daily_orders': orders_ts.mean(),
        'max_daily_orders': orders_ts.max()
    }
    
    return metrics, revenue_ma7, revenue_ma30, revenue_growth_daily

# Analyze stationarity
daily_data = time_aggregations['daily']
revenue_ts = daily_data.set_index('date')['total_revenue']
orders_ts = daily_data.set_index('date')['unique_orders']

revenue_stationarity = analyze_stationarity(revenue_ts, "Daily Revenue")
orders_stationarity = analyze_stationarity(orders_ts, "Daily Orders")

# Calculate growth metrics
growth_metrics, revenue_ma7, revenue_ma30, revenue_growth = calculate_growth_metrics(daily_data)

print(f"\n💰 Business Growth Metrics:")
print(f"   • Average daily revenue: R$ {growth_metrics['avg_daily_revenue']:.2f}")
print(f"   • Revenue volatility (daily): {growth_metrics['revenue_volatility']:.3f}")
print(f"   • Trend slope: R$ {growth_metrics['trend_slope']:.2f} per day")
print(f"   • Revenue range: R$ {growth_metrics['revenue_range']:.2f}")
print(f"   • Peak daily revenue: R$ {growth_metrics['max_daily_revenue']:.2f}")
print(f"   • Peak daily orders: {growth_metrics['max_daily_orders']:,}")

# Trend direction analysis
trend_direction = "Growing" if growth_metrics['trend_slope'] > 0 else "Declining"
print(f"\n📈 Overall Business Trend: {trend_direction}")

if growth_metrics['trend_slope'] > 0:
    annual_growth_estimate = growth_metrics['trend_slope'] * 365
    print(f"   • Estimated annual revenue growth: R$ {annual_growth_estimate:,.2f}")

# Visualize trend analysis
plt.figure(figsize=(15, 10))

# Revenue with moving averages
plt.subplot(2, 2, 1)
plt.plot(revenue_ts.index, revenue_ts, alpha=0.5, label='Daily Revenue', color='lightblue')
plt.plot(revenue_ma7.index, revenue_ma7, label='7-day MA', color='blue', linewidth=2)
plt.plot(revenue_ma30.index, revenue_ma30, label='30-day MA', color='red', linewidth=2)
plt.title('Revenue Trend Analysis', fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Revenue (R$)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Growth rate distribution
plt.subplot(2, 2, 2)
revenue_growth_clean = revenue_growth.dropna()
plt.hist(revenue_growth_clean, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
plt.axvline(revenue_growth_clean.mean(), color='red', linestyle='--', 
           label=f'Mean: {revenue_growth_clean.mean():.3f}')
plt.title('Daily Revenue Growth Rate Distribution', fontweight='bold')
plt.xlabel('Growth Rate')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# Cumulative revenue
plt.subplot(2, 2, 3)
cumulative_revenue = revenue_ts.cumsum()
plt.plot(cumulative_revenue.index, cumulative_revenue, color='purple', linewidth=2)
plt.title('Cumulative Revenue Over Time', fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative Revenue (R$)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)

# Revenue vs Orders correlation
plt.subplot(2, 2, 4)
plt.scatter(orders_ts, revenue_ts, alpha=0.6, color='coral')
correlation = orders_ts.corr(revenue_ts)
plt.title(f'Daily Orders vs Revenue\n(Correlation: {correlation:.3f})', fontweight='bold')
plt.xlabel('Number of Orders')
plt.ylabel('Revenue (R$)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n🔄 Time Series Characteristics Summary:")
if revenue_stationarity:
    print(f"   • Revenue series: {'Stationary' if revenue_stationarity['is_stationary'] else 'Non-stationary'}")
if orders_stationarity:
    print(f"   • Orders series: {'Stationary' if orders_stationarity['is_stationary'] else 'Non-stationary'}")
print(f"   • Revenue-Orders correlation: {correlation:.3f}")
print(f"   • Business trend: {trend_direction} at R$ {growth_metrics['trend_slope']:.2f}/day")

## 5. Customer Temporal Behavior Analysis

Analyze how different customer segments behave over time.

In [ ]:
# Customer Temporal Behavior Analysis
print("👥 Customer Temporal Behavior Analysis")
print("=" * 40)

def analyze_customer_temporal_patterns(data):
    """
    Analyze temporal patterns in customer behavior
    """
    # Customer lifecycle analysis
    customer_timeline = data.groupby('customer_id').agg({
        'order_date': ['min', 'max', 'count'],
        'total_order_value': ['sum', 'mean'],
        'order_hour': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
        'order_dow': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0],
        'order_month': lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.iloc[0]
    }).reset_index()
    
    # Flatten column names
    customer_timeline.columns = ['customer_id', 'first_order', 'last_order', 'total_orders',
                                'total_spent', 'avg_order_value', 'preferred_hour', 
                                'preferred_dow', 'preferred_month']
    
    # Calculate customer lifespan
    customer_timeline['customer_lifespan_days'] = (
        customer_timeline['last_order'] - customer_timeline['first_order']
    ).dt.days
    
    # Time since last order (from end of analysis period)
    analysis_end = data['order_date'].max()
    customer_timeline['days_since_last_order'] = (
        analysis_end - customer_timeline['last_order']
    ).dt.days
    
    # Customer segments based on temporal behavior
    def classify_temporal_segment(row):
        total_orders = row['total_orders']
        lifespan = row['customer_lifespan_days']
        days_since_last = row['days_since_last_order']
        
        if total_orders == 1:
            return 'One-time Buyer'
        elif lifespan == 0:  # All orders on same day
            return 'Single-day Multi-buyer'
        elif lifespan <= 30 and total_orders > 1:
            return 'Quick Repeat Customer'
        elif lifespan > 30 and total_orders > 3:
            return 'Long-term Loyal'
        elif days_since_last > 180:
            return 'Dormant Customer'
        else:
            return 'Regular Customer'
    
    customer_timeline['temporal_segment'] = customer_timeline.apply(classify_temporal_segment, axis=1)
    
    return customer_timeline

# Perform customer temporal analysis
customer_temporal = analyze_customer_temporal_patterns(ts_df)

print(f"📊 Customer Temporal Analysis Results:")
print(f"   • Customers analyzed: {len(customer_temporal):,}")
print(f"   • Average customer lifespan: {customer_temporal['customer_lifespan_days'].mean():.1f} days")
print(f"   • Average orders per customer: {customer_temporal['total_orders'].mean():.1f}")

# Temporal segment distribution
segment_distribution = customer_temporal['temporal_segment'].value_counts()
print(f"\n🎯 Customer Temporal Segments:")
for segment, count in segment_distribution.items():
    percentage = (count / len(customer_temporal)) * 100
    print(f"   • {segment}: {count:,} customers ({percentage:.1f}%)")

# Segment performance analysis
segment_performance = customer_temporal.groupby('temporal_segment').agg({
    'total_spent': ['mean', 'sum'],
    'avg_order_value': 'mean',
    'total_orders': 'mean',
    'customer_lifespan_days': 'mean',
    'days_since_last_order': 'mean'
}).round(2)

# Flatten columns
segment_performance.columns = ['avg_total_spent', 'total_revenue', 'avg_order_value', 
                              'avg_orders', 'avg_lifespan', 'avg_days_since_last']

print(f"\n📈 Temporal Segment Performance:")
display(segment_performance.sort_values('total_revenue', ascending=False))

# Preferred timing analysis by segment
print(f"\n⏰ Preferred Timing by Customer Segment:")

# Hour preferences
hour_preferences = customer_temporal.groupby('temporal_segment')['preferred_hour'].apply(
    lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.mean()
)

# Day preferences
dow_preferences = customer_temporal.groupby('temporal_segment')['preferred_dow'].apply(
    lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else x.mean()
)

day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for segment in segment_distribution.index:
    preferred_hour = hour_preferences.get(segment, 0)
    preferred_dow = int(dow_preferences.get(segment, 0))
    preferred_day = day_names[preferred_dow] if 0 <= preferred_dow < 7 else 'Unknown'
    
    print(f"   • {segment}:")
    print(f"     - Preferred hour: {preferred_hour:.0f}:00")
    print(f"     - Preferred day: {preferred_day}")

# Customer acquisition trends
monthly_acquisition = customer_temporal.groupby(
    customer_temporal['first_order'].dt.to_period('M')
).size().reset_index()
monthly_acquisition.columns = ['month', 'new_customers']

print(f"\n📅 Customer Acquisition Trends:")
if len(monthly_acquisition) > 1:
    acquisition_growth = (
        (monthly_acquisition['new_customers'].iloc[-1] - monthly_acquisition['new_customers'].iloc[0]) /
        monthly_acquisition['new_customers'].iloc[0] * 100
    )
    print(f"   • Customer acquisition growth: {acquisition_growth:+.1f}% (first to last month)")
    print(f"   • Peak acquisition month: {monthly_acquisition.loc[monthly_acquisition['new_customers'].idxmax(), 'month']}")
    print(f"   • Average monthly acquisition: {monthly_acquisition['new_customers'].mean():.0f} customers")

In [ ]:
# Customer Cohort Analysis
print("👥 Customer Cohort Analysis")
print("=" * 30)

def create_cohort_analysis(data):
    """
    Create customer cohort analysis based on first purchase month
    """
    # Define cohorts based on first purchase month
    customer_cohorts = data.groupby('customer_id')['order_date'].min().reset_index()
    customer_cohorts.columns = ['customer_id', 'first_purchase_date']
    customer_cohorts['cohort_month'] = customer_cohorts['first_purchase_date'].dt.to_period('M')
    
    # Merge back with transaction data
    data_with_cohorts = data.merge(customer_cohorts, on='customer_id')
    data_with_cohorts['order_period'] = data_with_cohorts['order_date'].dt.to_period('M')
    
    # Calculate period number (months since first purchase)
    data_with_cohorts['period_number'] = (
        data_with_cohorts['order_period'] - data_with_cohorts['cohort_month']
    ).apply(attrgetter('n'))
    
    # Cohort table creation
    cohort_data = data_with_cohorts.groupby(['cohort_month', 'period_number'])['customer_id'].nunique().reset_index()
    cohort_table = cohort_data.pivot(index='cohort_month', columns='period_number', values='customer_id')
    
    # Calculate cohort sizes
    cohort_sizes = customer_cohorts.groupby('cohort_month')['customer_id'].nunique()
    
    # Calculate retention rates
    cohort_retention = cohort_table.divide(cohort_sizes, axis=0)
    
    return cohort_table, cohort_retention, cohort_sizes

try:
    from operator import attrgetter
    
    # Perform cohort analysis
    cohort_counts, cohort_retention, cohort_sizes = create_cohort_analysis(ts_df)
    
    print(f"📊 Cohort Analysis Results:")
    print(f"   • Cohort periods analyzed: {len(cohort_sizes)}")
    print(f"   • Largest cohort: {cohort_sizes.max():,} customers")
    print(f"   • Average cohort size: {cohort_sizes.mean():.0f} customers")
    
    # Display cohort retention rates
    print(f"\n📈 Cohort Retention Rates (showing first 6 months):")
    if cohort_retention.shape[1] > 0:
        display(cohort_retention.iloc[:, :min(6, cohort_retention.shape[1])].round(3))
    
    # Calculate average retention by period
    if cohort_retention.shape[1] > 1:
        avg_retention = cohort_retention.mean()
        print(f"\n🎯 Average Retention Rates by Period:")
        for period, retention in avg_retention.head(6).items():
            if pd.notna(retention):
                print(f"   • Month {period}: {retention:.1%}")
    
    # Visualize cohort heatmap if we have sufficient data
    if cohort_retention.shape[0] > 2 and cohort_retention.shape[1] > 2:
        plt.figure(figsize=(12, 8))
        
        # Plot cohort heatmap
        sns.heatmap(
            cohort_retention.iloc[:, :min(12, cohort_retention.shape[1])],
            annot=True,
            fmt='.2%',
            cmap='YlOrRd',
            linewidths=0.5
        )
        
        plt.title('Customer Cohort Retention Rates', fontsize=16, fontweight='bold')
        plt.xlabel('Period Number (Months since first purchase)')
        plt.ylabel('Cohort Month')
        plt.tight_layout()
        plt.show()
    
    else:
        print(f"\n📊 Insufficient data for cohort heatmap visualization")
        print(f"   (Need at least 3 cohorts and 3 periods)")

except Exception as e:
    print(f"Error in cohort analysis: {e}")
    print("This might be due to data limitations or dependencies.")
    
    # Alternative simple retention analysis
    print(f"\n📊 Alternative Customer Retention Analysis:")
    
    # Simple repeat purchase analysis
    customer_order_counts = ts_df.groupby('customer_id').size()
    repeat_customers = (customer_order_counts > 1).sum()
    total_customers = len(customer_order_counts)
    repeat_rate = repeat_customers / total_customers
    
    print(f"   • Total customers: {total_customers:,}")
    print(f"   • Repeat customers: {repeat_customers:,}")
    print(f"   • Overall repeat rate: {repeat_rate:.1%}")
    
    # Customer lifetime analysis
    customer_lifetime = customer_temporal['customer_lifespan_days'].describe()
    print(f"\n📅 Customer Lifetime Statistics (days):")
    print(f"   • Average: {customer_lifetime['mean']:.1f}")
    print(f"   • Median: {customer_lifetime['50%']:.1f}")
    print(f"   • 75th percentile: {customer_lifetime['75%']:.1f}")
    print(f"   • Maximum: {customer_lifetime['max']:.0f}")

## 6. Business Forecasting and Predictive Analytics

Apply forecasting techniques for business planning and prediction.

In [ ]:
# Business Forecasting and Predictive Analytics
print("🔮 Business Forecasting and Predictive Analytics")
print("=" * 50)

def create_simple_forecasts(daily_data, forecast_days=30):
    """
    Create simple forecasting models for business planning
    """
    forecasts = {}
    
    # Prepare data
    ts_data = daily_data.set_index('date')['total_revenue'].fillna(method='ffill')
    
    if len(ts_data) < 10:
        print("Insufficient data for forecasting")
        return None
    
    print(f"📊 Creating forecasts for {forecast_days} days ahead:")
    print(f"   • Historical data points: {len(ts_data)}")
    print(f"   • Data range: {ts_data.index.min().date()} to {ts_data.index.max().date()}")
    
    # 1. Moving Average Forecast
    ma_window = min(7, len(ts_data) // 2)  # Use 7 days or half the data, whichever is smaller
    ma_forecast = ts_data.rolling(ma_window).mean().iloc[-1]
    forecasts['moving_average'] = ma_forecast
    
    # 2. Linear Trend Forecast
    days_numeric = np.arange(len(ts_data))
    trend_slope, trend_intercept = np.polyfit(days_numeric, ts_data, 1)
    
    future_days = np.arange(len(ts_data), len(ts_data) + forecast_days)
    trend_forecast = trend_slope * future_days + trend_intercept
    forecasts['linear_trend'] = trend_forecast
    
    # 3. Seasonal Naive Forecast (use same day of week from previous week)
    if len(ts_data) >= 7:
        seasonal_forecast = []
        for i in range(forecast_days):
            # Look back 7 days for seasonal pattern
            lookback_idx = max(0, len(ts_data) - 7 + (i % 7))
            seasonal_forecast.append(ts_data.iloc[lookback_idx])
        forecasts['seasonal_naive'] = np.array(seasonal_forecast)
    
    # 4. Exponential Smoothing (simple)
    alpha = 0.3  # Smoothing parameter
    exp_smooth = [ts_data.iloc[0]]
    
    for i in range(1, len(ts_data)):
        exp_smooth.append(alpha * ts_data.iloc[i] + (1 - alpha) * exp_smooth[-1])
    
    exp_forecast = exp_smooth[-1]  # Use last smoothed value for forecast
    forecasts['exponential_smoothing'] = exp_forecast
    
    return forecasts, ts_data, trend_slope, trend_intercept

def evaluate_forecast_accuracy(actual, predicted, method_name):
    """
    Evaluate forecast accuracy using multiple metrics
    """
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return {
        'method': method_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape
    }

# Create forecasts
daily_data = time_aggregations['daily']
forecast_results = create_simple_forecasts(daily_data, forecast_days=14)

if forecast_results is not None:
    forecasts, historical_data, trend_slope, trend_intercept = forecast_results
    
    print(f"\n🎯 Forecast Results (14-day outlook):")
    
    # Display forecast values
    print(f"   • Moving Average: R$ {forecasts['moving_average']:.2f}/day")
    print(f"   • Linear Trend (Day 14): R$ {forecasts['linear_trend'][-1]:.2f}/day")
    if 'seasonal_naive' in forecasts:
        print(f"   • Seasonal Naive (avg): R$ {forecasts['seasonal_naive'].mean():.2f}/day")
    print(f"   • Exponential Smoothing: R$ {forecasts['exponential_smoothing']:.2f}/day")
    
    # Business implications
    current_avg = historical_data.tail(7).mean()
    ma_change = ((forecasts['moving_average'] - current_avg) / current_avg) * 100
    
    print(f"\n📈 Business Implications:")
    print(f"   • Current 7-day average: R$ {current_avg:.2f}/day")
    print(f"   • Forecast vs current: {ma_change:+.1f}% change")
    print(f"   • Monthly revenue projection: R$ {forecasts['moving_average'] * 30:,.2f}")
    
    if trend_slope > 0:
        print(f"   • Trend indicates growth: +R$ {trend_slope:.2f}/day")
    else:
        print(f"   • Trend indicates decline: R$ {trend_slope:.2f}/day")
    
    # Visualize forecasts
    plt.figure(figsize=(15, 10))
    
    # Historical data
    plt.subplot(2, 1, 1)
    plt.plot(historical_data.index, historical_data.values, 
             label='Historical Revenue', color='blue', alpha=0.7)
    
    # Add trend line
    trend_line = trend_slope * np.arange(len(historical_data)) + trend_intercept
    plt.plot(historical_data.index, trend_line, 
             label=f'Trend Line (slope: R${trend_slope:.2f}/day)', color='red', linestyle='--')
    
    plt.title('Historical Revenue with Trend Analysis', fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Daily Revenue (R$)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    # Forecast visualization
    plt.subplot(2, 1, 2)
    
    # Last 30 days of historical data
    recent_data = historical_data.tail(30)
    plt.plot(recent_data.index, recent_data.values, 
             label='Recent Historical', color='blue', linewidth=2)
    
    # Create future dates
    last_date = historical_data.index[-1]
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=14, freq='D')
    
    # Plot forecasts
    plt.axhline(y=forecasts['moving_average'], color='green', linestyle='-', 
               label=f'Moving Average: R${forecasts["moving_average"]:.2f}')
    
    plt.plot(future_dates, forecasts['linear_trend'], 
             label='Linear Trend', color='red', marker='o', linestyle='-')
    
    if 'seasonal_naive' in forecasts:
        plt.plot(future_dates, forecasts['seasonal_naive'], 
                 label='Seasonal Naive', color='orange', marker='s', linestyle='-')
    
    plt.axhline(y=forecasts['exponential_smoothing'], color='purple', linestyle=':', 
               label=f'Exp. Smoothing: R${forecasts["exponential_smoothing"]:.2f}')
    
    plt.title('14-Day Revenue Forecasts', fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Daily Revenue (R$)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

else:
    print("Unable to generate forecasts due to insufficient data")

In [ ]:
# Comprehensive Business Planning Insights
print("📋 Comprehensive Business Planning Insights")
print("=" * 45)

def generate_business_planning_insights(time_aggregations, customer_temporal, forecast_results):
    """
    Generate comprehensive insights for business planning
    """
    insights = {
        'seasonal_patterns': {},
        'customer_insights': {},
        'operational_recommendations': [],
        'growth_opportunities': [],
        'risk_factors': []
    }
    
    # Seasonal patterns
    monthly_data = time_aggregations['monthly']
    if len(monthly_data) > 1:
        peak_month = monthly_data.loc[monthly_data['total_revenue'].idxmax(), 'month']
        low_month = monthly_data.loc[monthly_data['total_revenue'].idxmin(), 'month']
        seasonality_ratio = monthly_data['total_revenue'].max() / monthly_data['total_revenue'].min()
        
        insights['seasonal_patterns'] = {
            'peak_month': str(peak_month),
            'low_month': str(low_month),
            'seasonality_ratio': seasonality_ratio,
            'seasonal_strength': 'High' if seasonality_ratio > 2 else 'Moderate' if seasonality_ratio > 1.5 else 'Low'
        }
    
    # Customer insights
    loyal_customers = len(customer_temporal[customer_temporal['temporal_segment'] == 'Long-term Loyal'])
    dormant_customers = len(customer_temporal[customer_temporal['temporal_segment'] == 'Dormant Customer'])
    total_customers = len(customer_temporal)
    
    insights['customer_insights'] = {
        'loyal_percentage': (loyal_customers / total_customers) * 100,
        'dormant_percentage': (dormant_customers / total_customers) * 100,
        'avg_customer_lifespan': customer_temporal['customer_lifespan_days'].mean(),
        'repeat_rate': len(customer_temporal[customer_temporal['total_orders'] > 1]) / total_customers * 100
    }
    
    # Generate recommendations
    # Operational recommendations
    hourly_data = time_aggregations['hourly']
    peak_hour = hourly_data.loc[hourly_data['unique_orders'].idxmax(), 'hour']
    
    insights['operational_recommendations'] = [
        f"Staff peak operations during {peak_hour}:00-{peak_hour+1}:00 for maximum efficiency",
        f"Implement inventory planning based on {insights['seasonal_patterns'].get('seasonal_strength', 'moderate')} seasonality",
        "Focus customer service resources during identified peak periods",
        "Optimize delivery logistics for peak demand days"
    ]
    
    # Growth opportunities
    insights['growth_opportunities'] = [
        f"Re-engage {dormant_customers:,} dormant customers through targeted campaigns",
        f"Develop loyalty programs for {loyal_customers:,} long-term loyal customers",
        "Expand marketing during low-season periods to smooth demand",
        "Implement cross-selling during peak shopping periods"
    ]
    
    # Risk factors
    daily_data = time_aggregations['daily']
    revenue_volatility = daily_data['total_revenue'].std() / daily_data['total_revenue'].mean()
    
    insights['risk_factors'] = [
        f"Revenue volatility: {'High' if revenue_volatility > 0.5 else 'Moderate' if revenue_volatility > 0.3 else 'Low'} (CV: {revenue_volatility:.2f})",
        f"Customer concentration: Monitor {insights['customer_insights']['loyal_percentage']:.1f}% loyal customer dependency",
        "Seasonal demand fluctuations require careful inventory management",
        "Customer acquisition trends need monitoring for sustainable growth"
    ]
    
    return insights

# Generate comprehensive insights
business_insights = generate_business_planning_insights(
    time_aggregations, customer_temporal, forecast_results
)

# Display comprehensive business planning insights
print(f"\n🎯 COMPREHENSIVE BUSINESS PLANNING INSIGHTS")
print(f"=" * 55)

print(f"\n📅 SEASONAL PATTERNS:")
seasonal = business_insights['seasonal_patterns']
if seasonal:
    print(f"   • Peak season: {seasonal['peak_month']}")
    print(f"   • Low season: {seasonal['low_month']}")
    print(f"   • Seasonality strength: {seasonal['seasonal_strength']} (ratio: {seasonal['seasonality_ratio']:.2f})")
else:
    print(f"   • Insufficient data for seasonal analysis")

print(f"\n👥 CUSTOMER INSIGHTS:")
customer = business_insights['customer_insights']
print(f"   • Customer loyalty rate: {customer['loyal_percentage']:.1f}%")
print(f"   • Customer repeat rate: {customer['repeat_rate']:.1f}%")
print(f"   • Average customer lifespan: {customer['avg_customer_lifespan']:.1f} days")
print(f"   • Dormant customers: {customer['dormant_percentage']:.1f}% (reactivation opportunity)")

print(f"\n⚙️ OPERATIONAL RECOMMENDATIONS:")
for i, rec in enumerate(business_insights['operational_recommendations'], 1):
    print(f"   {i}. {rec}")

print(f"\n📈 GROWTH OPPORTUNITIES:")
for i, opp in enumerate(business_insights['growth_opportunities'], 1):
    print(f"   {i}. {opp}")

print(f"\n⚠️ RISK FACTORS TO MONITOR:")
for i, risk in enumerate(business_insights['risk_factors'], 1):
    print(f"   {i}. {risk}")

# Strategic recommendations summary
print(f"\n\n🎯 STRATEGIC RECOMMENDATIONS SUMMARY")
print(f"=" * 45)

print(f"\n🥇 HIGH PRIORITY (Immediate - 1 month):")
print(f"   • Optimize staffing for peak hours and days")
print(f"   • Launch dormant customer reactivation campaign")
print(f"   • Implement revenue forecasting for inventory planning")

print(f"\n🥈 MEDIUM PRIORITY (2-3 months):")
print(f"   • Develop seasonal marketing strategies")
print(f"   • Create loyalty program for repeat customers")
print(f"   • Enhance customer acquisition during low seasons")

print(f"\n🥉 LONG-TERM PRIORITIES (3-6 months):")
print(f"   • Build predictive analytics capabilities")
print(f"   • Implement advanced customer segmentation")
print(f"   • Develop dynamic pricing strategies")

print(f"\n✅ Time Series Analysis Complete!")
print(f"   Advanced EDA with Business Intelligence trilogy finished.")
print(f"   Ready for implementation and continuous monitoring.")

## Summary - Time Series Patterns in Order Data

### What We've Accomplished

1. **✅ Comprehensive Time Series Data Loading**: Processed 35,000+ temporal records with advanced time-based features
2. **✅ Multi-Scale Temporal Aggregation**: Created daily, weekly, monthly, and hourly aggregations for pattern analysis
3. **✅ Advanced Seasonal Decomposition**: Applied statistical decomposition to identify trends, seasonality, and residuals
4. **✅ Customer Temporal Behavior Analysis**: Segmented customers based on temporal patterns and lifecycle stages
5. **✅ Cohort Analysis**: Analyzed customer retention and behavior evolution over time
6. **✅ Business Forecasting**: Created multiple forecasting models for 14-day revenue predictions
7. **✅ Strategic Business Planning**: Generated comprehensive insights for operational and strategic planning

### Key Business Insights Discovered

**Temporal Patterns:**
- Peak business hours, days, and seasonal periods identified
- Revenue volatility and trend analysis for planning
- Weekend vs weekday behavioral differences

**Customer Lifecycle Insights:**
- Customer temporal segmentation (One-time, Loyal, Dormant, etc.)
- Retention patterns and cohort analysis
- Customer acquisition trends and timing preferences

**Forecasting and Planning:**
- Multiple forecasting approaches for different business scenarios
- Growth trend analysis and revenue projections
- Risk assessment and opportunity identification

### Advanced Techniques Mastered

- **Seasonal Decomposition**: STL decomposition for trend and seasonality analysis
- **Stationarity Testing**: Augmented Dickey-Fuller tests for time series properties
- **Cohort Analysis**: Customer retention and lifecycle analysis
- **Multiple Forecasting Methods**: Moving average, linear trend, seasonal naive, exponential smoothing
- **Business Intelligence Integration**: Translating statistical insights into actionable strategies

### Complete Advanced EDA Framework

**Part 1:** Customer Behavior Analysis and Segmentation
**Part 2:** Product Performance Metrics and Insights  
**Part 3:** Time Series Patterns in Order Data

This comprehensive framework provides a systematic approach to advanced exploratory data analysis that combines customer analytics, product intelligence, and temporal insights for complete business understanding.

### Next Steps
- Implement real-time monitoring dashboards
- Build automated forecasting pipelines
- Develop advanced machine learning models
- Create executive reporting systems

## 🎯 Practice Exercises - Time Series Analysis

Master temporal analytics techniques:

1. **Advanced Forecasting**: Implement ARIMA or exponential smoothing models for more sophisticated predictions

2. **Anomaly Detection**: Create algorithms to detect unusual patterns in daily revenue or order volumes

3. **Seasonal Strategy Planning**: Develop detailed seasonal marketing and inventory strategies based on patterns

4. **Customer Lifetime Value Modeling**: Use temporal patterns to predict customer lifetime value

5. **Real-time Monitoring System**: Design a system to monitor key temporal metrics in real-time

In [ ]:
# Exercise Space - Time Series Analysis
# Use this space to practice the temporal analytics techniques

# Exercise 1: Advanced Forecasting
# Implement ARIMA or other sophisticated forecasting models

# Exercise 2: Anomaly Detection
# Create algorithms to detect unusual temporal patterns

# Exercise 3: Seasonal Strategy Planning
# Develop comprehensive seasonal business strategies

# Exercise 4: Customer Lifetime Value Modeling
# Use temporal patterns for CLV prediction

# Exercise 5: Real-time Monitoring System
# Design real-time temporal metrics monitoring